
## 1) Setup
- Works for **SQLite**: `.db`, `.sqlite`, `.sqlite3`
- If you don't know the exact path, the cell will try to **auto-detect** a file that starts with `chicago` and has a SQLite extension.



# Chicago Crime — How many tables are in the dataset?
This notebook helps you **count tables** in a Chicago Crime dataset, typically provided as a **SQLite database** (e.g., `chicago_crime.db`).  
It also includes an optional section for **CSV-based datasets**.

> If you run this on Kaggle or your own Jupyter, just set the `DB_PATH` (for SQLite) or `CSV_DIR` (for CSVs) and run the cells.


In [1]:

from pathlib import Path
import sqlite3
import pandas as pd

# === CONFIG ===
# If you already know the DB path, put it here, e.g.:
# DB_PATH = Path('/home/jovyan/work/chicago_crime.db')
DB_PATH = None  # will attempt auto-detect below

# Optional: folder with CSVs (if your dataset is CSV-based)
# CSV_DIR = Path('/home/jovyan/work/chicago_crime_csv')
CSV_DIR = None

# === AUTO-DETECT A SQLITE FILE ===
if DB_PATH is None:
    candidates = []
    for ext in ('.db', '.sqlite', '.sqlite3'):
        candidates += list(Path('.').rglob(f'chicago*{ext}'))
    DB_PATH = candidates[0] if candidates else None

DB_PATH



## 2) Count tables in SQLite
This uses the system catalog `sqlite_master` to list and count tables (ignoring internal `sqlite_%` tables).


In [ ]:

def list_sqlite_tables(db_path: Path):
    if db_path is None or not Path(db_path).exists():
        print("❌ SQLite file not found. Set DB_PATH manually to a valid file.")
        return None, None
    conn = sqlite3.connect(str(db_path))
    try:
        # list tables
        tables = pd.read_sql(
            "SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%' ORDER BY name;",
            conn
        )['name'].tolist()
        # count tables
        count = pd.read_sql(
            "SELECT COUNT(*) AS n FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%';",
            conn
        )['n'].iloc[0]
        return tables, count
    finally:
        conn.close()

tables, n_tables = list_sqlite_tables(DB_PATH)
if tables is not None:
    print(f"SQLite file: {DB_PATH}")
    print(f"Tables ({n_tables}):")
    for t in tables:
        print(" -", t)
else:
    print("No SQLite file detected in current tree. Set DB_PATH to your chicago_crime.db (or similar).")



### SQL-only (if you're answering inside an SQL tool)
```sql
-- Count tables:
SELECT COUNT(*) AS n
FROM sqlite_master
WHERE type = 'table' AND name NOT LIKE 'sqlite_%';

-- List table names:
SELECT name
FROM sqlite_master
WHERE type = 'table' AND name NOT LIKE 'sqlite_%'
ORDER BY name;
```



## 3) (Optional) If your dataset is a folder of CSVs
Sometimes Kaggle datasets ship as multiple CSV files. In that case, each CSV can be treated as a "table".
Set `CSV_DIR` above and run this cell to count/list them.


In [ ]:

from pathlib import Path

def list_csv_tables(csv_dir: Path):
    if csv_dir is None:
        print("CSV_DIR is not set.")
        return None, None
    csv_dir = Path(csv_dir)
    if not csv_dir.exists():
        print(f"❌ CSV directory not found: {csv_dir}")
        return None, None
    files = sorted([p for p in csv_dir.glob('*.csv')])
    return files, len(files)

csv_files, n_csv = list_csv_tables(CSV_DIR)
if csv_files is not None:
    print(f"CSV directory: {CSV_DIR}")
    print(f"CSV 'tables' ({n_csv}):")
    for f in csv_files:
        print(" -", f.name)
